# Вебинар 6. Двухуровневые модели рекомендаций


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()
n_user_before = data_train_lvl_1['user_id'].nunique()

#параметры функции могут отличаться
data_train_lvl_1 = prefilter_items(data_train_lvl_1, take_n_popular=5000, item_features=None)

n_items_after = data_train_lvl_1['item_id'].nunique()
n_user_after = data_train_lvl_1['user_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))
print('Decreased # users from {} to {}'.format(n_user_before, n_user_after))

Decreased # items from 83685 to 5001
Decreased # users from 2498 to 2498


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, N=5)

[1082185, 1005186, 916122, 981760, 937791]

In [6]:
recommender.get_own_recommendations(2375, N=5)

[957420, 867138, 1005186, 983936, 947147]

In [7]:
recommender.get_similar_items_recommendation(2375, N=5)

[868764, 889731, 1055646, 1046545, 845208]

In [8]:
recommender.get_similar_users_recommendation(2375, N=5)

[820321, 824180, 1005637, 974204, 906509]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [18]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [19]:
N = 50
users_train = data_train_lvl_1['user_id'].tolist()
users_valid = result_lvl_1['user_id'].tolist()
new_users = list(set(users_valid) - set(users_train))
users_list = list(set(users_valid) & set(users_train))

cols = ['top_popular', 'als_recommendations', 'own_recommendations', 'similar_items_recommendation',
        'similar_users_recommendation']
df = pd.DataFrame(index=users_valid, columns=cols)

overall_top_purchases = recommender.overall_top_purchases[:N]

for user in users_valid:
    for col in cols:
        df.loc[user, col] = overall_top_purchases 
        
for user in users_list:
    df.loc[user, 'top_popular'] = overall_top_purchases
    df.loc[user, 'als_recommendations'] = recommender.get_als_recommendations(user, N)
    df.loc[user, 'own_recommendations'] = recommender.get_own_recommendations(user, N)
    df.loc[user, 'similar_items_recommendation'] = recommender.get_similar_items_recommendation(user, N)    
    df.loc[user, 'similar_users_recommendation'] = recommender.get_similar_users_recommendation(user, N)  
            
df = df.reset_index()
df.columns = ['user_id'] + cols
result_lvl_1 = result_lvl_1.merge(df, on='user_id', how='left')
result_lvl_1.head(2)

,user_id,actual,top_popular,als_recommendations,own_recommendations,similar_items_recommendation,similar_users_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1082185, 6534178, 1029743, 995242, 1106523, 9...","[1008625, 1138508, 878026, 848669, 970365, 998...","[1008625, 966826, 1082185, 6534178, 1029743, 9...","[824758, 999999, 1082185, 1098066, 1124432, 94...","[893729, 856166, 918598, 931108, 855356, 94339..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1082185, 6534178, 1029743, 995242, 1106523, 9...","[1082185, 6034857, 860776, 1127831, 1058997, 9...","[904833, 921920, 853235, 980657, 820701, 89068...","[1137346, 1133018, 5569845, 1106523, 985999, 8...","[860439, 879922, 890344, 978313, 888431, 82785..."


In [21]:
result_lvl_1['own_recommendtions_top_popular'] = result_lvl_1['own_recommendations'] + df['top_popular']
len(result_lvl_1.own_recommendtions_top_popular[0])

100

In [22]:
for col in result_lvl_1.columns[2:]:    
    recall = result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], N), axis=1).mean()
    print(f'recall_at_{len(result_lvl_1[col][0])} {col}: {round(recall, 4)}')

recall_at_50 top_popular: 0.0921
recall_at_50 als_recommendations: 0.0431
recall_at_50 own_recommendations: 0.0098
recall_at_50 similar_items_recommendation: 0.058
recall_at_50 similar_users_recommendation: 0.004
recall_at_100 own_recommendtions_top_popular: 0.0098


**Ответ:**
- А) Совмещение own recommendtions + top-popular recall лучше не стал

In [23]:
k = [20, 50, 100, 200, 500]
cols = ['own_recommendations_' + str(i) for i in k]
df = pd.DataFrame(index=users_valid, columns=cols)

for i, N in enumerate(k):
    overall_top_purchases = recommender.overall_top_purchases[:N]
    for user in new_users:        
        df.loc[user, cols[i]] = overall_top_purchases
        
    for user in users_list:
        df.loc[user, cols[i]] = recommender.get_own_recommendations(user, N)
        
df = df.reset_index()
df.columns = ['user_id'] + cols
result_lvl_1 = result_lvl_1.merge(df, on='user_id', how='left')

In [24]:
for i, N in enumerate(k):   
    recall = result_lvl_1.apply(lambda row: recall_at_k(row[cols[i]], row['actual'], N), axis=1).mean()
    print(f'recall_at_{N} {cols[i]}: {round(recall, 4)}')

recall_at_20 own_recommendations_20: 0.0021
recall_at_50 own_recommendations_50: 0.0098
recall_at_100 own_recommendations_100: 0.0475
recall_at_200 own_recommendations_200: 0.1205
recall_at_500 own_recommendations_500: 0.2228


**Ответ:** 
- В)* При увеличении количества кандидатов recall@k увеличивается 
- C)* Все очень индивидуально. Зависит от рекомендуемых item, категории users, способа рекомендаций.

### Задание 2.

Обучите модель 2-ого уровня, при этом:  
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар  
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2  
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [25]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']
result_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [26]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2.head(4)

,user_id,candidates
0,2070,"[935578, 880427, 970890, 910473, 995816, 97476..."
1,2021,"[840386, 840609, 1114992, 996114, 990688, 8217..."
2,1753,"[966891, 972312, 1033913, 819840, 843464, 9017..."
3,2120,"[918598, 879922, 1033913, 982989, 989275, 9552..."


In [27]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1

users_lvl_2.head(4)

,user_id,item_id,drop
0,2070,935578,1
0,2070,880427,1
0,2070,970890,1
0,2070,910473,1


In [28]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)
targets_lvl_2.head(2)

,user_id,item_id,target
0,2070,935578,0.0
1,2070,880427,0.0


In [29]:
targets_lvl_2['target'].mean()

0.01623649653923281

In [30]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,935578,0.0,1194,GROCERY,National,REFRGRATD JUICES/DRNKS,DAIRY CASE 100% PURE JUICE - O,89 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,880427,0.0,1882,GROCERY,National,BAKED BREAD/BUNS/ROLLS,PREMIUM BREAD,24 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [31]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [32]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [33]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
CPU times: user 10.4 s, sys: 104 ms, total: 10.5 s
Wall time: 484 ms


In [34]:
recall_at_k(train_preds, y_train)

0.9837635034607672

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 0.27%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 